In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [118]:
import pandas as pd
import xgboost as xgb
import pickle

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [32]:
df = pd.read_csv('/content/drive/MyDrive/MSC DSAI/Python/Project/air_pollution_data.csv')
df.head()

,city,date,aqi,co,no,no2,o3,so2,pm2_5,pm10,nh3
0,Ahmedabad,30-11-2020,5,520.71,2.38,16.28,130.18,47.68,65.96,72.13,8.36
1,Ahmedabad,01-12-2020,5,1682.28,7.71,54.84,0.73,21.70,120.95,154.53,27.36
2,Ahmedabad,02-12-2020,5,1815.80,16.54,49.35,0.17,23.84,133.47,172.63,28.12
3,Ahmedabad,03-12-2020,5,2296.45,41.57,40.10,0.00,35.76,150.37,202.15,36.48
4,Ahmedabad,04-12-2020,5,2189.64,23.92,58.95,0.02,28.13,160.79,205.80,40.53


In [33]:
df.shape

(23504, 11)

In [6]:
df.isnull().sum()

,0
city,0
date,0
aqi,0
co,0
no,0
no2,0
o3,0
so2,0
pm2_5,0
pm10,0


In [11]:
df['city'].value_counts()

,count
Ahmedabad,1
Aizawl,1
Thiruvananthapuram,1
Talcher,1
Shillong,1
Patna,1
Mumbai,1
Lucknow,1
Kolkata,1
Kochi,1


In [35]:
"""df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df.head()"""

"df['date'] = pd.to_datetime(df['date'])\ndf['year'] = df['date'].dt.year\ndf.head()"

In [36]:
le = LabelEncoder()

In [37]:
df['city'] = le.fit_transform(df['city'])
df['date'] = le.fit_transform(df['date'])
df.head()

,city,date,aqi,co,no,no2,o3,so2,pm2_5,pm10,nh3
0,0,881,5,520.71,2.38,16.28,130.18,47.68,65.96,72.13,8.36
1,0,27,5,1682.28,7.71,54.84,0.73,21.70,120.95,154.53,27.36
2,0,57,5,1815.80,16.54,49.35,0.17,23.84,133.47,172.63,28.12
3,0,87,5,2296.45,41.57,40.10,0.00,35.76,150.37,202.15,36.48
4,0,117,5,2189.64,23.92,58.95,0.02,28.13,160.79,205.80,40.53


In [41]:
df.corr()

,city,date,aqi,co,no,no2,o3,so2,pm2_5,pm10,nh3
city,1.000000e+00,4.182952e-15,0.047848,0.075597,0.039724,0.074986,0.033103,0.149843,0.102874,0.097101,-0.069039
date,4.182952e-15,1.000000e+00,-0.007867,-0.016546,-0.009080,-0.013259,0.002407,-0.009879,-0.013387,-0.013923,-0.008344
aqi,4.784839e-02,-7.867248e-03,1.000000,0.377355,0.173879,0.440113,0.085184,0.303629,0.456002,0.467122,0.361675
co,7.559681e-02,-1.654623e-02,0.377355,1.000000,0.843799,0.805820,-0.313232,0.631510,0.929881,0.935345,0.603286
no,3.972433e-02,-9.079883e-03,0.173879,0.843799,1.000000,0.499226,-0.267879,0.532777,0.719160,0.718655,0.416600
no2,7.498615e-02,-1.325931e-02,0.440113,0.805820,0.499226,1.000000,-0.380578,0.708760,0.737977,0.755135,0.608555
o3,3.310318e-02,2.407033e-03,0.085184,-0.313232,-0.267879,-0.380578,1.000000,-0.163729,-0.191523,-0.203824,-0.192751
so2,1.498426e-01,-9.878861e-03,0.303629,0.631510,0.532777,0.708760,-0.163729,1.000000,0.552935,0.562396,0.468437
pm2_5,1.028743e-01,-1.338718e-02,0.456002,0.929881,0.719160,0.737977,-0.191523,0.552935,1.000000,0.990648,0.558785
pm10,9.710062e-02,-1.392258e-02,0.467122,0.935345,0.718655,0.755135,-0.203824,0.562396,0.990648,1.000000,0.586979


In [103]:
X = df.drop(columns = ['aqi'])
X.head()

,city,date,co,no,no2,o3,so2,pm2_5,pm10,nh3
0,0,881,520.71,2.38,16.28,130.18,47.68,65.96,72.13,8.36
1,0,27,1682.28,7.71,54.84,0.73,21.70,120.95,154.53,27.36
2,0,57,1815.80,16.54,49.35,0.17,23.84,133.47,172.63,28.12
3,0,87,2296.45,41.57,40.10,0.00,35.76,150.37,202.15,36.48
4,0,117,2189.64,23.92,58.95,0.02,28.13,160.79,205.80,40.53


In [104]:
y = df['aqi']
y.head()

,aqi
0,5
1,5
2,5
3,5
4,5


In [105]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [106]:
model1 = LinearRegression()
model1.fit(X_train, y_train)

LinearRegression()

In [107]:
y_pred1 = model1.predict(X_test)
print("MAE:",mean_absolute_error(y_test, y_pred1))
print("MSE:",mean_squared_error(y_test, y_pred1))
print("R2:",r2_score(y_test, y_pred1))

MAE: 0.9508639902511976
MSE: 1.3467161351414179
R2: 0.3300789026933958


In [108]:
model2 = DecisionTreeRegressor()
model2.fit(X_train, y_train)

DecisionTreeRegressor()

In [109]:
y_pred2 = model2.predict(X_test)
print("MAE:",mean_absolute_error(y_test, y_pred2))
print("MSE:",mean_squared_error(y_test, y_pred2))
print("R2:",r2_score(y_test, y_pred2))

MAE: 0.09423526909168262
MSE: 0.09423526909168262
R2: 0.9531228644050858


In [110]:
model3 = RandomForestRegressor()
model3.fit(X_train, y_train)

RandomForestRegressor()

In [111]:
y_pred3 = model3.predict(X_test)
print("MAE:",mean_absolute_error(y_test, y_pred3))
print("MSE:",mean_squared_error(y_test, y_pred3))
print("R2:",r2_score(y_test, y_pred3))

MAE: 0.10459263986385874
MSE: 0.051700212720697726
R2: 0.9742818383673723


In [112]:
model4 = xgb.XGBRegressor()
model4.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [113]:
y_pred4 = model4.predict(X_test)
print("MAE:",mean_absolute_error(y_test, y_pred4))
print("MSE:",mean_squared_error(y_test, y_pred4))
print("R2:",r2_score(y_test, y_pred4))

MAE: 0.1185431716239749
MSE: 0.0547907186896928
R2: 0.9727444726999116


In [114]:
model5 = svm.SVR()
model5.fit(X_train, y_train)

SVR()

In [115]:
y_pred5 = model5.predict(X_test)
print("MAE:",mean_absolute_error(y_test, y_pred5))
print("MSE:",mean_squared_error(y_test, y_pred5))
print("R2:",r2_score(y_test, y_pred5))

MAE: 0.39223065946223695
MSE: 0.30873825817690276
R2: 0.8464188054918632


In [122]:
with open('rf.pkl', 'wb') as file:
  pickle.dump(model3, file)

In [124]:
with open('xgb.pkl', 'wb') as file:
  pickle.dump(model4, file)